In [1]:
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
author_id={
  "Достоевский Ф. М.": 9150,
  "Роллинс Дж.": 59396,
  "Фицджеральд Ф. С.": 28727,
  "Глуховский Д. А.": 53427,
  "Стругацкий А. Н.": 26268,
  "Лукьяненко С. В.": 16626,
  "Фрай М.": 28927,
  "Хантер Э.": 37969,
  "Роулинг Дж. К.": 104832
}

In [3]:
def loger(func):
    import functools
    @functools.wraps(func)
    def wraper(*argc,**argv):
        res = func(*argc,**argv)
        with mutex:
            global n_processed
            n_processed += 1
            if n_processed % 10 == 0:
                print(f"\r{n_processed} objects are processed...", end='', flush=True)
        return res
    return wraper

In [4]:
@loger
def extract_book_info(book_id):
    import re
    import requests
    from bs4 import BeautifulSoup
    from functools import reduce 
    
    url='https://www.moscowbooks.ru'+book_id[0]
    html=requests.get(url).text
    soup=BeautifulSoup(html,'lxml')
    
    ans=dict()
    
    ans['ID']=re.search('\d+',book_id[0])[0]
    ans['Автор']=soup.find(class_="page-header__author").text[1:-1]
    ans['Название']=book_id[1]
    ans['Наличие']='true' if soup.find(class_="book__shop-instock") is None else 'false'
    tmp=soup.find(class_="book__price")
    ans['Цена']=''.join(re.findall(r'\d',tmp.text))+' '+re.search(r'\w*\.',tmp.text)[0]
    
    description = soup.find(class_="book__description")
    for d in description.find_all(['b','a']):
        d.replace_with('')

    for br in description.find_all("br"):
        br.replace_with("\n")

    ans['Описание']=re.sub('\s+', ' ', description.text)[1:]
    
    
    
    ans['Рейтинг']=soup.find(class_="book___rating-stars rating-stars rating-stars_lg")['data-rate']
    ans['Обложка']='https://www.moscowbooks.ru'+soup.find(class_="book__img book__img_default gallery__img")['src']
    ans['Стикеры']=reduce(lambda x,y:x+';'+y,{i.text for i in soup.find_all(class_="label")},'')[1:]
   
    
    
    
    
    
    for side in ['left','right']:
        for dl in soup.find(class_=f'book__details-{side}').find_all('dl'):
            dt=dl.find_all('dt')
            key=re.search('.*:',re.sub('\s+', ' ', dt[0].text)[1:])[0][:-1]
            value=re.sub('\s+', ' ', dt[1].text)[1:]
            ans[key]=value
    
    return ans

In [5]:
def f(author):
    from bs4 import BeautifulSoup
    import requests
    url_0 = f'https://www.moscowbooks.ru/catalog/author/{author_id[author]}/'
    html_0 = requests.get(url_0).text
    soup = BeautifulSoup(html_0, 'lxml')
    ans=[]
    number_of_pages=max(len(soup.find_all(class_='pager__text')),2)
    for i in range(1,number_of_pages):
        url=url_0+f'?page={i}'
        html=requests.get(url).text
        soup=BeautifulSoup(html,'lxml')
        ans+=[(i['href'],i.text) for i in soup.find_all(class_="book-preview__title-link")]
    return ans

In [6]:
if __name__ == '__main__':
    from multiprocessing.pool import ThreadPool as Pool
    from functools import reduce 
    
    with Pool(9) as p:
        x=p.map(f,author_id )
    x=[element for sublist in x for element in sublist]

In [9]:
if __name__ == '__main__':
    from multiprocessing.pool import ThreadPool as Pool
    from threading import Lock
    import pandas as pd
    
    mutex = Lock()
    n_processed = 0
    
    with Pool(15) as p:
        result = p.map(extract_book_info, x)
    df = pd.DataFrame(result)
    df.sort_values(by=['ID'], inplace=True)

    with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
        df.to_csv(f_csv, index=False)

240 objects are processed...